In [1]:
from sagas.ofbiz.builder import oc, finder, abbrev

In [2]:
print(oc.delegator.getDelegatorName())

default


In [3]:
import graphene

class Person(graphene.ObjectType):
    first_name = graphene.String()
    last_name = graphene.String()
    full_name = graphene.String()

    def resolve_full_name(self, info):
        return '{} {}'.format(self.first_name, self.last_name)

⊕ [Graphene-Python](http://docs.graphene-python.org/en/latest/types/mutations/)


In [5]:
class CreatePerson(graphene.Mutation):
    class Arguments:
        name = graphene.String()

    ok = graphene.Boolean()
    person = graphene.Field(lambda: Person)

    def mutate(self, info, name):
        person = Person(name=name)
        ok = True
        return CreatePerson(person=person, ok=ok)

# ... the Mutation Class

class Person(graphene.ObjectType):
    name = graphene.String()
    age = graphene.Int()

class MyMutations(graphene.ObjectType):
    create_person = CreatePerson.Field()

# We must define a query for our schema
class Query(graphene.ObjectType):
    person = graphene.Field(Person)

schema = graphene.Schema(query=Query, mutation=MyMutations)    

## InputFields and InputObjectTypes


In [6]:
import graphene

class Person(graphene.ObjectType):
    name = graphene.String()
    age = graphene.Int()
    
class PersonInput(graphene.InputObjectType):
    name = graphene.String(required=True)
    age = graphene.Int(required=True)

class CreatePerson(graphene.Mutation):
    class Arguments:
        person_data = PersonInput(required=True)

    person = graphene.Field(lambda: Person)
    Output = Person

    @staticmethod
    def mutate(root, info, person_data=None):
        person = Person(
            name=person_data.name,
            age=person_data.age
        )
        # return CreatePerson(person=person)
        return person

# ... the Mutation Class    
class MyMutations(graphene.ObjectType):
    create_person = CreatePerson.Field()

# We must define a query for our schema
class Query(graphene.ObjectType):
    person = graphene.Field(Person)

schema = graphene.Schema(query=Query, mutation=MyMutations)    

In [7]:
import json

q1 = '''
mutation myFirstMutation {
    createPerson(personData: {name:"Peter", age: 24}) {
        person {
            name,
            age
        }
    }
}
'''.strip()
q2 = '''
mutation myFirstMutation {
    createPerson(personData: {name:"Peter", age: 24}) {
        name
        __typename
    }
}
'''.strip()
# result = schema.execute(q1)
result = schema.execute(q2)
print(json.dumps(result.data, indent=2, ensure_ascii=False))

{
  "createPerson": {
    "name": "Peter",
    "__typename": "Person"
  }
}


In [2]:
import uuid
from sagas.ofbiz.builder import abbrev

def rand_id():
    return abbrev("f_"+str(uuid.uuid4()), 15)

print(uuid.uuid4())
strval="4affa0dc-(var)-474c-b019-6c9946d6d6ef"
print(strval.replace("(var)", rand_id()))

6dc16232-0a0a-41dc-b61a-aee713a05fff
4affa0dc-f_bb520235-50df..-474c-b019-6c9946d6d6ef


In [6]:
from sagas.ofbiz.builder import oc, finder, abbrev
from sagas.util.str_converters import to_camel_case, to_snake_case
import graphene
import json

def input_to_dictionary(input, entity, of_type):
    output=of_type()
    m = oc.j.HashMap()
    for key in input:        
        value=input[key]
        setattr(output, key, value)
        m[to_camel_case(key)] = value
    val=oc.delegator.create(entity, m)
    # print(to_snake_case('lastUpdatedStamp'), val['lastUpdatedStamp'], val)
    setattr(output, "last_updated_stamp", val['lastUpdatedStamp'])
    return output

class TestingType(graphene.ObjectType):
    testing_type_id = graphene.String()
    last_updated_stamp=graphene.String()
    
val=input_to_dictionary({"testing_type_id":rand_id()},
                    "TestingType", TestingType)
print(val.last_updated_stamp)

2019-01-23 01:35:48.097


In [8]:
class TestingTypeInput(graphene.InputObjectType):
    testing_type_id = graphene.String(required=True)

class CreateTestingType(graphene.Mutation):
    class Arguments:
        testing_type_data = TestingTypeInput(required=True)

    testing_type = graphene.Field(lambda: TestingType)
    Output = TestingType

    @staticmethod
    def mutate(root, info, testing_type_data=None):
        testing_type = input_to_dictionary(testing_type_data, "TestingType", TestingType)
        # return CreatePerson(person=person)
        return testing_type

# ... the Mutation Class    
class MyMutations(graphene.ObjectType):
    create_testing_type = CreateTestingType.Field()

# We must define a query for our schema
class Query(graphene.ObjectType):
    testing_type = graphene.Field(TestingType)

schema = graphene.Schema(query=Query, mutation=MyMutations)   

## testing
q2 = '''
mutation myFirstMutation {
    createTestingType(testingTypeData: {testingTypeId:"(id)"}) {
        testingTypeId
        lastUpdatedStamp
        __typename
    }
}
'''.replace("(id)", rand_id()).strip()
# result = schema.execute(q1)
result = schema.execute(q2)
print(json.dumps(result.data, indent=2, ensure_ascii=False))

{
  "createTestingType": {
    "testingTypeId": "f_cdb89977-33c4..",
    "lastUpdatedStamp": "2019-01-23 01:37:05.448",
    "__typename": "TestingType"
  }
}


In [13]:
from tabulate import tabulate

def desc_model(entity_name):
    ent = oc.delegator.getModelEntity(entity_name)
    autoflds=ent.getAutomaticFieldNames()
    print(ent.getPlainTableName(), ent.getTitle(), ent.getDescription())
    table_header = ['name', 'type', 'string']
    table_data = []
    names = ent.getAllFieldNames()
    for field_name in names:
        if field_name not in autoflds:
            fld = ent.getField(field_name)
            table_data.append((abbrev(field_name, 20),
                               fld.getType(),
                               abbrev(fld.getColName(), 25)))
    print(tabulate(table_data, headers=table_header, tablefmt='psql'))
    # desc_relations(entity_name)

desc_model("TestingType")

TESTING_TYPE Testing Entity Type None
+---------------+-------------+-----------------+
| name          | type        | string          |
|---------------+-------------+-----------------|
| testingTypeId | id          | TESTING_TYPE_ID |
| description   | description | DESCRIPTION     |
+---------------+-------------+-----------------+
